In [76]:
import numpy as np
import torch
import os
import glob

# Define the root directory
root_dir = "/home/lala/other/data/sIMphar_synth/"

# Function to process each directory recursively
def process_directory(directory):
    # Get all files named "results.npy" in the directory
    file_paths = glob.glob(os.path.join(directory, "results.npy"))

    # Process each file in the directory
    for file_path in file_paths:
        # Load data from the file
        x = np.load(file_path, allow_pickle=True)

        # Create a directory to save tensors if it doesn't exist
        save_dir = os.path.dirname(file_path)

        distances_tensors = []

        for motion_data in x.item()["motion"]:
            distances = []
            for frame in motion_data.transpose(2, 0, 1):
                # Extract coordinates for joint 21 and joint 22 for the current frame
                joint_21 = frame[20]  # Index 20 corresponds to joint 21 (0-based indexing)
                joint_22 = frame[21]  # Index 21 corresponds to joint 22 (0-based indexing)

                # Calculate the Euclidean distance between joint 21 and joint 22 using NumPy
                distance = np.linalg.norm(joint_21 - joint_22)

                # Append the distance to the list of distances for this motion
                distances.append(distance)

            # Convert distances list to a PyTorch tensor
            distances_tensor = torch.tensor(distances)

            # Append the tensor to the list of tensors
            distances_tensors.append(distances_tensor)

        # Save each tensor to a separate file in the same directory
        for idx, tensor in enumerate(distances_tensors):
            save_path = os.path.join(save_dir, f'distances_tensor_motion_{idx}.pt')
            torch.save(tensor, save_path)

    # Recursively process subdirectories
    subdirectories = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(directory, subdirectory)
        process_directory(subdirectory_path)

# Start processing from the root directory
process_directory(root_dir)
